In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import scrapy
import random
import time
from urllib.parse import urljoin
import re
from tqdm import tqdm

In [ ]:
def get_url_from_google(index):
	def ajusta_url(url):
    # Define um padrão para ".com/" ou ".com.br/"
		pattern = r"^(https?://[^/]+?(?:\.com(?:\.br)?/))"
		match = re.match(pattern, url)
		if match:
			return match.group(1)
		else: return url


	query = "+".join(base['name_normalized'].iloc[index].split())
	search_url = f"https://www.google.com/search?q={query}"

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	response = requests.get(search_url, headers=headers)

	soup = BeautifulSoup(response.text, "html.parser")
	

	main_data=soup.find_all("a")
	#pattern_inverso = rf'href="(https?://[^"]*{'google'}[^"]*\.(com|org)(?:\.br)?[^"]*)'
	#print(main_data)

	keyword = base["name_normalized_cortado"][index]
	pattern = rf'href="(https://.*{keyword}.*\.com/)"'

	link_primeiro_site_encontrado = []
	for keyword in base['name_normalized_cortado'][index]:
		#print(keyword)
		
		pattern = rf'href="(https?://[^"]*{keyword}[^"]*\.(com|org)(?:\.br)?[^"]*)'
		#pattern = rf'href="(https?://[^"]*{keyword}[^"]*\.(com|org)(?:\.br)?/?)"'
		for i in main_data[:21]:
			print(i)
			match = re.search(pattern, str(i))
			if match:
				#print("URL encontrada:", match.group())
				if match.group() not in link_primeiro_site_encontrado:
					if match.group(1) is not None:
						link_primeiro_site_encontrado.append(ajusta_url(match.group(1)))
						
		if link_primeiro_site_encontrado == [] and len(main_data) > 14:
			for element in [8,15]:
				href = main_data[element].get('href')
				if href and len(href) < 100:
					link_primeiro_site_encontrado.append(ajusta_url(href))
		if link_primeiro_site_encontrado == []:
			for i in main_data[8:21]:
				pattern_1 = rf'href="(https?://[^"]*{keyword}[^"]*[^/]*?)"'
				match = re.search(pattern_1, str(i))
				if match:
						link_primeiro_site_encontrado.append(match.group(1))

		# else:
		# 	href = main_data[8].get('href')
		# 	link_primeiro_site_encontrado.append(ajusta_url(href))
			
	return link_primeiro_site_encontrado


def pega_url_de_lista_com_timesleep(inicio, termino, base):
	fim = dict()
	for index in tqdm(range(inicio, termino), desc="Scraping"):
		try:
			url = get_url_from_google(index)
			#print(f"URL encontrada para '{site}': {url}")
			fim[base.iloc[index]['name']] = url
			# Pausa aleatória entre as requisições
			time.sleep(random.uniform(3, 15))
		except Exception as e:
			print(f"Erro ao processar o índice {index}: {e}")
		
	
	return fim

In [22]:
def get_url_from_agencia_brasil(index):
	

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		for i in range(1, 50):
				
			search_url = f"https://busca.ebc.com.br/nodes?page={i}utf8=%E2%9C%93&q={query}&site_id=agenciabrasil"
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			print(links)
			lista.append(links)

		return lista
	

	def encontra_links_do_html(soup):
		main_data=soup.find_all("a")
		#print(main_data)
		pattern_agencia_brasil = rf'href="(http://agenciabrasil.*)"'
		link_sites_encontrados = []

		for i in main_data:
			match = re.search(pattern_agencia_brasil, str(i))
			#print(i)
			if match:
				#print(f'deu match:{match}')
				#print("URL encontrada:", match.group())
				if match.group() not in link_sites_encontrados:
					if match.group(1) is not None:
						link_sites_encontrados.append(match.group(1))
		return link_sites_encontrados

	#pattern_inverso = rf'href="(https?://[^"]*{'google'}[^"]*\.(com|org)(?:\.br)?[^"]*)'
	#print(main_data)

	#keyword = base["name_normalized_cortado"][index]
	#pattern = rf'href="(https://.*{keyword}.*\.com/)"'


	# link_primeiro_site_encontrado = []
	# for keyword in base['name_normalized_cortado'][index]:
	# 	#print(keyword)
		
	# 	pattern = rf'href="(https?://[^"]*{keyword}[^"]*\.(com|org)(?:\.br)?[^"]*)'
	# 	#pattern = rf'href="(https?://[^"]*{keyword}[^"]*\.(com|org)(?:\.br)?/?)"'
	# 	for i in main_data[:21]:
	# 		print(i)
	# 		match = re.search(pattern, str(i))
	# 		if match:
	# 			#print("URL encontrada:", match.group())
	# 			if match.group() not in link_primeiro_site_encontrado:
	# 				if match.group(1) is not None:
	# 					link_primeiro_site_encontrado.append(ajusta_url(match.group(1)))
						
	# 	if link_primeiro_site_encontrado == [] and len(main_data) > 14:
	# 		for element in [8,15]:
	# 			href = main_data[element].get('href')
	# 			if href and len(href) < 100:
	# 				link_primeiro_site_encontrado.append(ajusta_url(href))
	# 	if link_primeiro_site_encontrado == []:
	# 		for i in main_data[8:21]:
	# 			pattern_1 = rf'href="(https?://[^"]*{keyword}[^"]*[^/]*?)"'
	# 			match = re.search(pattern_1, str(i))
	# 			if match:
	# 					link_primeiro_site_encontrado.append(match.group(1))

	# 	# else:
	# 	# 	href = main_data[8].get('href')
	# 	# 	link_primeiro_site_encontrado.append(ajusta_url(href))
			
	return main(index)

def pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave):
	fim = dict()
	for chave in tqdm(palavras_chave, desc="Scraping"):
		url = get_url_from_agencia_brasil(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
		
		
	
	return fim

palavras_chave = ['b3']

pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave)

Scraping:   0%|          | 0/1 [00:00<?, ?it/s]

['http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-cai-para-r-580-apos-trump-falar-em-acordo-com-china" style="display:none" target="_blank', 'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-cai-para-r-580-apos-trump-falar-em-acordo-com-china" target="_blank', 'http://agenciabrasil.ebc.com.br/educacao/noticia/2025-04/programa-que-oferta-bolsas-de-estudo-negros-tem-inscricao-ate-dia-30" style="display:none" target="_blank', 'http://agenciabrasil.ebc.com.br/educacao/noticia/2025-04/programa-que-oferta-bolsas-de-estudo-negros-tem-inscricao-ate-dia-30" target="_blank', 'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-chega-perto-dos-r-590-com-tensoes-comerciais" style="display:none" target="_blank', 'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-chega-perto-dos-r-590-com-tensoes-comerciais" target="_blank', 'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/venda-de-veiculos-financiados-tem-queda-em-marco-diz-b3" style="di

Scraping: 100%|██████████| 1/1 [00:35<00:00, 35.18s/it]


{'b3': [['http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-cai-para-r-580-apos-trump-falar-em-acordo-com-china" style="display:none" target="_blank',
   'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-cai-para-r-580-apos-trump-falar-em-acordo-com-china" target="_blank',
   'http://agenciabrasil.ebc.com.br/educacao/noticia/2025-04/programa-que-oferta-bolsas-de-estudo-negros-tem-inscricao-ate-dia-30" style="display:none" target="_blank',
   'http://agenciabrasil.ebc.com.br/educacao/noticia/2025-04/programa-que-oferta-bolsas-de-estudo-negros-tem-inscricao-ate-dia-30" target="_blank',
   'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-chega-perto-dos-r-590-com-tensoes-comerciais" style="display:none" target="_blank',
   'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-chega-perto-dos-r-590-com-tensoes-comerciais" target="_blank',
   'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/venda-de-veiculos-financiados-tem-queda-